In [ ]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re


In [ ]:

#torch.cuda.empty_cache()

In [ ]:
path="/home/vicuna-weights-7B"
peft_path='/home/medical-llama/saved_models-news'
tokenizer = LlamaTokenizer.from_pretrained(path)

In [ ]:
with open("/home/medical-llama/saved_models-news/adapter_config.json", 'r') as json_file:
    config = json.load(json_file)

In [ ]:
peft_config = PeftConfig.from_pretrained(peft_path)

In [ ]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
model = PeftModel.from_pretrained(
        model,
        peft_path,
        torch_dtype=torch.float16,
        
)

In [ ]:
# set these properly to avoid errors.
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [ ]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops=[], encounters=1):
        super().__init__()
        self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

In [ ]:
stop_words_ids = [torch.tensor([835]).to('cuda'),
                          torch.tensor([2277, 29937]).to('cuda')]  # '###' can be encoded in two different ways.
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [ ]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
### Instruction:
[INSTRUCTION]
 
### Response:
"""

def create_prompt(instruction) :
    return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)


def generate_response(prompt, model: PeftModel) :
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to('cuda')
 
    generation_config = GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=1,
        min_length=1,
        max_time=3,
        
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=5,
            stopping_criteria=stopping_criteria,
        )
    

def format_response(response) :
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))


def ask_vicuna(prompt, model=model):
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    b=format_response(response)
    a=b.split('###')
    return a[0]

In [ ]:
def pubmed_evaluator(Question, Context):
    prompt_template=f"""Task Description:

    In this task, you'll be provided with a research question along with a context from a PubMed abstract. Your goal is to answer the research question based on the information given in the context. The answers will be strictly restricted to 'yes', 'no', or 'maybe'.

    'Yes' should be used when the information provided in the context clearly supports the posed question.
    'No' should be used when the information provided in the context clearly contradicts the posed question.
    'Maybe' should be used when the information provided in the context is ambiguous, not enough, or has conflicting data in regard to the posed question.
    The question will be related to a medical or biomedical topic.

    Question:
    {Question}

    Context:
    {Context}

    Remember, your response should be strictly 'yes', 'no', or 'maybe' based on the given context.

    Answer: """
    
    return prompt_template

In [ ]:
ask_vicuna(pubmed_evaluator())